In [92]:
from sqlalchemy import create_engine
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json

# Staging
server_stg = '34.95.134.241'
database_stg = 'VoistonDb'
username_stg = 'sqlserver'
password_stg = '.KN_mWXA7kxsnat'

# Prod
server_prod = '34.95.157.73'
database_prod = 'VoistonDb'
username_prod = 'sqlserver'
password_prod = 'BF2y4d!.DqR6'
# String de conexão com SQLAlchemy

engine_stg = create_engine(f"mssql+pyodbc://{username_stg}:{password_stg}@{server_stg}/{database_stg}?driver=ODBC+Driver+17+for+SQL+Server")
engine_prod = create_engine(f"mssql+pyodbc://{username_prod}:{password_prod}@{server_prod}/{database_prod}?driver=ODBC+Driver+17+for+SQL+Server")

In [93]:
# Codigo para pegar os PartnerIDs de stg e entao ler em prod:
patient_ids_list = [
    280530, 280543, 280544, 280547, 280549, 280550, 280551, 280554, 280555, 
    280545, 280546, 280548, 280552, 280553, 280556, 280557, 280558, 280559, 
    280560, 280561, 280562, 280563, 280564, 280565, 280566, 280567, 280568, 
    280569, 280570, 280571, 280531
]

patient_ids_sql = ", ".join(map(str, patient_ids_list))

query_partner_ids = f"""
SELECT DISTINCT PartnerID
FROM EHRs e
WHERE e.Patient_ID IN ({patient_ids_sql})
"""
partner_ids = (
    pd.read_sql(query_partner_ids, engine_stg)["PartnerID"]
    .dropna()
    .astype(str)
    .unique()
    .tolist()
)

In [94]:
# Proximo passo: pegar os EHRs com os Contains em stg e rodar o codigo de flag bits:
FLAGS = [
    ("DiagnosisOphthalmological", 1), ("DiagnosisComorbidities", 2), ("FamiliarHistory", 4),
    ("PreviousOphthalmologicalProcedures", 8), ("MedicationsInUse", 16), ("MedicationsPrescribed", 32),
    ("SlitLamp", 64), ("NeurossensorialOcularEvaluation", 128), ("IntraocularPressure", 256),
    ("VisualAcuity", 512), ("Refraction", 1024), ("Cid", 2048), ("MedicalConduct", 4096),
    ("RefractionOnly", 8192), ("ExaminationReport", 16384), ("SurgicalDescription", 32768),
    ("AnesthesiaRecord", 65536), ("MedicalLeaveCertificate", 131072), ("MedicalReport", 262144),
    ("PreopEvaluation", 524288), ("InformedConsent", 1048576), ("PatientWish", 2097152),
    ("TreatmentContraindication", 4194304),
]
flag_names = [name for name, _ in FLAGS]

select_bits = ",\n    ".join(
    [
        f"CAST(CASE WHEN ([e].[Contains] & {val}) = {val} THEN 1 ELSE 0 END AS int) AS [{name}]"
        for name, val in FLAGS
    ]
)

query_ehrs_stg = f"""
SELECT
    p.Name AS PatientName,
    p.ID AS Patient_ID,
    e.ID AS EHR_ID,
    e.PartnerID,
    e.FormattedText,
    COALESCE(e.DateTaken, e.DateCreated) AS EHR_Date,
    e.[Contains],
    {select_bits}
FROM EHRs e
JOIN Patients p ON e.Patient_ID = p.ID
WHERE e.Patient_ID IN ({patient_ids_sql})
"""
df_ehrs = pd.read_sql(query_ehrs_stg, engine_stg)

In [95]:
df_ehrs.head()

,PatientName,Patient_ID,EHR_ID,PartnerID,FormattedText,EHR_Date,Contains,DiagnosisOphthalmological,DiagnosisComorbidities,FamiliarHistory,...,RefractionOnly,ExaminationReport,SurgicalDescription,AnesthesiaRecord,MedicalLeaveCertificate,MedicalReport,PreopEvaluation,InformedConsent,PatientWish,TreatmentContraindication
0,qareviewhosp_MARIA APARECIDA SILVA ANDRADE,280530,4641016,NP:664586_APAC:1506666_OCM:1096443,Consulta Médica\r\nID da Consulta: 1096443\r\n...,2016-11-16 13:41:07,5634,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,qareviewhosp_MARIA APARECIDA SILVA ANDRADE,280530,4641017,NP:664586_APAC:1506666_MR:386059,Receita\r\nID da Receita: 386059\r\nID do Aten...,2016-11-16 13:41:07,9216,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,qareviewhosp_MARIA APARECIDA SILVA ANDRADE,280530,4641018,NP:664586_APAC:3392246_OCM:2829195,Consulta Médica\r\nID da Consulta: 2829195\r\n...,2023-05-26 11:16:10,4931,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,qareviewhosp_MARIA APARECIDA SILVA ANDRADE,280530,4641019,NP:664586_APAC:3392246_OCM:2829392,Consulta Médica\r\nID da Consulta: 2829392\r\n...,2023-05-26 11:16:10,1280,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,qareviewhosp_MARIA APARECIDA SILVA ANDRADE,280530,4641020,NP:664586_APAC:3392246_PEE:665080,Solicitações / Procedimentos\r\nID do Pedido: ...,2023-05-26 11:16:10,4096,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
ehr_flags = df_ehrs.melt(
    id_vars=["EHR_ID", "Patient_ID", "PartnerID", "EHR_Date", "FormattedText", "Contains"],
    value_vars=flag_names,
    var_name="ClassificationType",
    value_name="IsPresent",
)
ehr_flags = ehr_flags[ehr_flags["IsPresent"] == 1].drop(columns="IsPresent")
ehr_flags.head()

,EHR_ID,Patient_ID,PartnerID,EHR_Date,FormattedText,Contains,ClassificationType
2,4641018,280530,NP:664586_APAC:3392246_OCM:2829195,2023-05-26 11:16:10,Consulta Médica\r\nID da Consulta: 2829195\r\n...,4931,DiagnosisOphthalmological
6,4641022,280530,NP:664586_APAC:3613060_OCM:3052867,2024-01-18 11:15:49,Consulta Médica\r\nID da Consulta: 3052867\r\n...,4675,DiagnosisOphthalmological
7,4641023,280530,NP:664586_APAC:3679792_OCM:3120820,2024-04-04 08:13:39,Consulta Médica\r\nID da Consulta: 3120820\r\n...,4099,DiagnosisOphthalmological
14,4641030,280530,NP:664586_APAC:3731417_OCM:3176097,2024-05-27 13:43:28,Consulta Médica\r\nID da Consulta: 3176097\r\n...,6489,DiagnosisOphthalmological
19,4641035,280530,NP:664586_APAC:3792698_OCM:3233179,2024-08-01 10:13:26,Consulta Médica\r\nID da Consulta: 3233179\r\n...,6145,DiagnosisOphthalmological


In [97]:
# --------------------------------------------
# 4) MEDIDAS e PRESCRIÇÕES — LLM
# --------------------------------------------
query_med_stg = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    CASE WHEN m.LLMPrompt_ID IS NULL THEN 'parser' ELSE 'llm' END AS extractionType,
    m.Type_ID AS Measurement_ID,
    m.Side,
    m.StringValue,
    m.Metadata,
    mt.Name AS MeasurementName
FROM Measurements m
JOIN EHRs e ON e.ID = m.EHR_ID
LEFT JOIN MeasurementTypes mt ON m.Type_ID = mt.ID
WHERE e.Patient_ID IN ({patient_ids_sql})
"""
df_meas_stg = pd.read_sql(query_med_stg, engine_stg)
df_meas_stg.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,StringValue,Metadata,MeasurementName
0,4641016,NP:664586_APAC:1506666_OCM:1096443,llm,44,2,20/20,None,Visual Acuity Snellen
1,4641016,NP:664586_APAC:1506666_OCM:1096443,llm,44,1,20/20,None,Visual Acuity Snellen
2,4641016,NP:664586_APAC:1506666_OCM:1096443,llm,495,2,+1.25 D,None,GlassPrescription.Sphere
3,4641016,NP:664586_APAC:1506666_OCM:1096443,llm,496,2,+1.00 D,None,GlassPrescription.Cylinder
4,4641016,NP:664586_APAC:1506666_OCM:1096443,llm,497,2,115°,None,GlassPrescription.Axis


In [98]:
query_rx_stg = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    CASE WHEN m.LLMPrompt_ID IS NULL THEN 'parser' ELSE 'llm' END AS extractionType,
    m.Medication_ID AS Measurement_ID,
    m.Side,
    m.Frequency,
    m.FrequencyInstructions,
    m.Duration,
    m.StartDay,
    m.MedicationDosage,
    m.Dosage,
    m.MedicationDosageUnit,
    m.RouteOfAdministration,
    m.Metadata
FROM Prescriptions m
JOIN EHRs e ON e.ID = m.EHR_ID
WHERE e.Patient_ID IN ({patient_ids_sql})
"""
df_rx_stg = pd.read_sql(query_rx_stg, engine_stg)
df_rx_stg.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,Frequency,FrequencyInstructions,Duration,StartDay,MedicationDosage,Dosage,MedicationDosageUnit,RouteOfAdministration,Metadata
0,4641041,NP:664586_APAC:3820478_MR:1738373,llm,10922,0,6,6/6h,7,0,0.0,1.0,0,150,None
1,4641031,NP:664586_APAC:3731417_MR:1684733,llm,10930,0,3,3/3H,5,0,0.0,1.0,0,150,None
2,4641031,NP:664586_APAC:3731417_MR:1684733,llm,10930,0,4,4/4H,5,5,0.0,1.0,0,150,None
3,4641031,NP:664586_APAC:3731417_MR:1684733,llm,10930,0,6,6/6H,5,10,0.0,1.0,0,150,None
4,4641031,NP:664586_APAC:3731417_MR:1684733,llm,10930,0,8,8/8H,5,15,0.0,1.0,0,150,None


In [99]:
query_proc_stg = f"""
SELECT
    pe.EHR_ID,
    e.PartnerID,
    CASE WHEN pe.LLMPrompt_ID IS NULL THEN 'parser' ELSE 'llm' END AS extractionType,
    pe.Procedure_ID,
    pe.ProcedureType_ID,
    pt.DisplayName,
    pe.Side,
    pe.PeriodSpan,
    pe.Metadata,
    pr.ProcedureDate,
    pe.DateCreated AS EvidenceCreated
FROM ProcedureEvidences pe
JOIN EHRs e ON pe.EHR_ID = e.ID
JOIN ProcedureTypes pt ON pe.ProcedureType_ID = pt.ID
LEFT JOIN Procedures pr ON pe.Procedure_ID = pr.ID
WHERE e.Patient_ID IN ({patient_ids_sql})
"""
df_proc_stg = pd.read_sql(query_proc_stg, engine_stg)
df_proc_stg.head()

,EHR_ID,PartnerID,extractionType,Procedure_ID,ProcedureType_ID,DisplayName,Side,PeriodSpan,Metadata,ProcedureDate,EvidenceCreated
0,4641629,NP:1276849_APAC:4040361_OCM:3480953,llm,2121360.0,6,FACO,2,2,None,2025-05-08 11:28:36,2025-09-04 17:39:59.923
1,4641629,NP:1276849_APAC:4040361_OCM:3480953,llm,2121362.0,51,LIO,2,2,None,2025-05-08 11:28:36,2025-09-04 17:39:59.933
2,4641780,NP:430401_APAC:3955929_OCM:3397111,llm,2121365.0,10,Ozurdex,3,0,None,2025-02-07 13:13:12,2025-09-04 17:40:39.223
3,4641780,NP:430401_APAC:3955929_OCM:3397111,llm,2121273.0,51,LIO,3,0,None,NaT,2025-09-04 17:40:39.240
4,4641030,NP:664586_APAC:3731417_OCM:3176097,llm,2121292.0,6,FACO,2,3,None,2024-05-24 13:43:28,2025-09-05 20:13:26.153


In [100]:
meas_IDs = pd.read_sql("""
SELECT *
FROM MeasurementTypes
""", engine_prod)
meas_IDs.head()

,ID,Name,Unit,AlternativeNames,AnatomyElementSpecific,SideSpecific,DateCreated,LastModified,MeasurementType_ID,ScaleMax,...,LimitMin,ScaleStep,DisplayName,Priority,Metadata,DisplaySeriesAs,DisplaySeriesColor,DashboardUnit,AlertMax,AlertMin
0,41,Average RNFL Thickness,100,None,False,True,2020-01-07 20:35:15.127,2020-01-07 20:35:15.127,NaN,150.00,...,0.0,50.0,Avg RNFL Thickness,7.0000,False,0,None,0,123.5,47.5
1,42,Average Superior RNFL,100,None,False,True,2020-01-07 20:35:15.127,2020-01-07 20:35:15.127,41.0,150.00,...,0.0,50.0,Avg Sup RNFL,7.1000,False,0,None,0,159.5,43.5
2,43,Average Inferior RNFL,100,None,False,True,2020-01-07 20:35:15.127,2020-01-07 20:35:15.127,41.0,150.00,...,0.0,50.0,Avg Inf RNFL,7.2000,False,0,None,0,180.0,28.0
3,44,Visual Acuity Snellen,109,"acuidade visual com correção, acuidadevisual c...",False,True,2020-01-07 20:35:15.127,2025-08-17 02:23:43.537,NaN,1.05,...,0.0,0.1,Acuidade Visual,1.0001,False,0,None,0,2.0,0.0
4,45,Visual Acuity Snellen No Correction,109,"acuidade visual sem correção, acuidadevisual s...",False,True,2020-01-07 20:35:15.127,2025-08-17 02:25:09.560,44.0,1.05,...,0.0,0.1,Acuidade Visual (sc),1.1000,False,0,None,0,2.0,0.0


In [101]:
# --------------------------------------------
# 5b) PROD (parser only)
# --------------------------------------------
# Gera a lista de PartnerIDs para consulta em prod, a partir do que geramos anteriormente:
partner_ids_str = ",".join([f"'{pid}'" for pid in partner_ids])

query_med_prod = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    m.Type_ID AS Measurement_ID,
    m.Side,
    m.StringValue,
    m.Metadata,
    mt.Name AS MeasurementName
FROM Measurements m
JOIN EHRs e ON e.ID = m.EHR_ID
LEFT JOIN MeasurementTypes mt ON m.Type_ID = mt.ID
WHERE e.PartnerID IN ({partner_ids_str})
    AND m.OriginType <> 1
"""
df_meas_prod = pd.read_sql(query_med_prod, engine_prod)
df_meas_prod.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,StringValue,Metadata,MeasurementName
0,9437266,NP:1276849_APAC:4102303_MR:1909805,parser,NaN,1,1.00,None,None
1,7376917,NP:1216134_APAC:3628083_OCM:3068480,parser,41.0,2,14 µm,"{""extractedText"":""14 mmHg"",""contextText"":""OD P...",Average RNFL Thickness
2,7376917,NP:1216134_APAC:3628083_OCM:3068480,parser,41.0,1,14 µm,"{""extractedText"":""14 mmHg"",""contextText"":""OE P...",Average RNFL Thickness
3,7376916,NP:1216134_APAC:3637781_OCM:3078435,parser,41.0,2,14 µm,"{""extractedText"":""14 mmHg"",""contextText"":""OD P...",Average RNFL Thickness
4,7376916,NP:1216134_APAC:3637781_OCM:3078435,parser,41.0,1,14 µm,"{""extractedText"":""14 mmHg"",""contextText"":""OE P...",Average RNFL Thickness


In [102]:
# Prescricoes em prod:
query_rx_prod = f"""
SELECT
    m.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    m.Medication_ID AS Measurement_ID,
    m.Side,
    m.Frequency,
    m.FrequencyInstructions,
    m.Duration,
    m.StartDay,
    m.MedicationDosage,
    m.Dosage,
    m.MedicationDosageUnit,
    m.RouteOfAdministration,
    m.Metadata
FROM Prescriptions m
JOIN EHRs e ON e.ID = m.EHR_ID
WHERE e.PartnerID IN ({partner_ids_str})
"""
df_rx_prod = pd.read_sql(query_rx_prod, engine_prod)
df_rx_prod.head()

,EHR_ID,PartnerID,extractionType,Measurement_ID,Side,Frequency,FrequencyInstructions,Duration,StartDay,MedicationDosage,Dosage,MedicationDosageUnit,RouteOfAdministration,Metadata
0,8510657,NP:1094013_APAC:3087727_MR:1288347,parser,8351,3,6,None,0,0,0.0,1.0,0,150,None
1,5958254,NP:1106519_APAC:3840834_MR:1750157,parser,10974,0,4,None,10,0,0.0,1.0,0,150,None
2,5958254,NP:1106519_APAC:3840834_MR:1750157,parser,14102,0,4,None,5,0,0.0,0.0,0,150,None
3,5958254,NP:1106519_APAC:3840834_MR:1750157,parser,14102,0,6,None,5,5,0.0,0.0,0,150,None
4,5958254,NP:1106519_APAC:3840834_MR:1750157,parser,14102,0,8,None,5,10,0.0,0.0,0,150,None


In [103]:
query_proc_prod = f"""
SELECT
    pe.EHR_ID,
    e.PartnerID,
    'parser' AS extractionType,
    pe.Procedure_ID,
    pe.ProcedureType_ID,
    pt.DisplayName,
    pe.Side,
    pe.PeriodSpan,
    pe.Metadata,
    pr.ProcedureDate,
    pe.DateCreated AS EvidenceCreated
FROM ProcedureEvidences pe
JOIN EHRs e ON pe.EHR_ID = e.ID
JOIN ProcedureTypes pt ON pe.ProcedureType_ID = pt.ID
LEFT JOIN Procedures pr ON pe.Procedure_ID = pr.ID
WHERE e.PartnerID IN ({partner_ids_str})
"""
df_proc_prod = pd.read_sql(query_proc_prod, engine_prod)
df_proc_prod.head()

,EHR_ID,PartnerID,extractionType,Procedure_ID,ProcedureType_ID,DisplayName,Side,PeriodSpan,Metadata,ProcedureDate,EvidenceCreated
0,5810542,NP:430401_APAC:3467587_OCM:2905776,parser,2265240.0,6,FACO,1,3,None,2023-08-08 13:27:45,2025-03-12 21:56:26.180
1,5810540,NP:430401_APAC:3516644_OCM:2955586,parser,2265241.0,6,FACO,2,95,None,2023-06-26 13:56:03,2025-03-12 21:56:26.180
2,5810539,NP:430401_APAC:3536181_OCM:2975449,parser,2265241.0,6,FACO,2,116,None,2023-06-26 13:56:03,2025-03-12 21:56:26.497
3,5810541,NP:430401_APAC:3474581_OCM:2913049,parser,2265240.0,6,FACO,1,10,None,2023-08-08 13:27:45,2025-03-12 21:56:28.423
4,5810565,NP:430401_APAC:3788298_OCM:3228759,parser,2265242.0,64,Capsulotomia-YAG,3,0,None,2024-07-27 10:32:01,2025-03-12 21:56:32.557


In [104]:
# Unificar medidas, prescrições e procedimentos
df_meas_all = pd.concat([df_meas_stg, df_meas_prod], ignore_index=True, sort=False)
df_rx_all = pd.concat([df_rx_stg, df_rx_prod], ignore_index=True, sort=False)
df_proc_all = pd.concat([df_proc_stg, df_proc_prod], ignore_index=True, sort=False)


In [105]:
def dedup_parser_rows(df: pd.DataFrame,
                      id_col: str,
                      name_col: str,
                      value_col: str,
                      etype_col: str,
                      ehr_col: str = "EHR_ID",
                      metadata_col: str = "Metadata",
                      side_col: str | None = None) -> pd.DataFrame:
    """
    Deduplica *apenas* registros do parser dentro do mesmo EHR,
    mantendo 1 por (EHR_ID, ID, Name, Value[, Side]), preferindo a linha com Metadata.
    """
    if df is None or df.empty:
        return df

    required = [id_col, name_col, value_col, etype_col, ehr_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Faltam colunas para dedupe: {missing}")

    d = df.copy()
    mask_parser = d[etype_col].astype(str).str.lower().eq("parser")
    pars = d.loc[mask_parser].copy()
    other = d.loc[~mask_parser].copy()

    def _s(x):  # string canon básica
        return ("" if pd.isna(x) else str(x)).strip()

    def _s_intlike(x):  # ex.: 2.0 -> '2'
        v = pd.to_numeric(x, errors="coerce")
        return "" if pd.isna(v) else str(int(v))

    # monta chave
    key_parts = [
        pars[ehr_col].astype("Int64").astype(str),
        pars[id_col].astype("Int64").astype(str),
        pars[name_col].map(_s),
        pars[value_col].map(_s),
    ]
    if side_col and side_col in pars.columns:
        key_parts.append(pars[side_col].map(_s_intlike))

    pars["_k"] = "||".join(key_parts) if isinstance(key_parts, str) else (
        key_parts[0] + "||" + key_parts[1] + "||" + key_parts[2] + "||" + key_parts[3] +
        (("||" + key_parts[4]) if len(key_parts) == 5 else "")
    )

    # preferir quem tem metadata não vazia
    pars["_keep_pref"] = pars.get(metadata_col, "").astype(str).str.len().gt(0)

    # ordena e deduplica
    pars = pars.sort_values(by=["_k", "_keep_pref"], ascending=[True, False])
    pars = pars.drop_duplicates(subset=["_k"], keep="first").drop(columns=["_k", "_keep_pref"])

    return pd.concat([other, pars], ignore_index=True, sort=False)


In [106]:
df_meas_all = dedup_parser_rows(
    df_meas_all,
    id_col="Measurement_ID",
    name_col="MeasurementName",
    value_col="StringValue",
    etype_col="extractionType",
    ehr_col="EHR_ID",
    metadata_col="Metadata",
    side_col="Side"
)


In [107]:
# Coagir Measurement_ID para inteiro quando possível (corrige LLM string vs int)
for _df in (df_meas_all, df_rx_all):
    _df["Measurement_ID"] = pd.to_numeric(_df["Measurement_ID"], errors="coerce").astype("Int64")

In [108]:
df_meas_all.columns.tolist()

['EHR_ID',
 'PartnerID',
 'extractionType',
 'Measurement_ID',
 'Side',
 'StringValue',
 'Metadata',
 'MeasurementName']

In [109]:
df_rx_all.columns.tolist()

['EHR_ID',
 'PartnerID',
 'extractionType',
 'Measurement_ID',
 'Side',
 'Frequency',
 'FrequencyInstructions',
 'Duration',
 'StartDay',
 'MedicationDosage',
 'Dosage',
 'MedicationDosageUnit',
 'RouteOfAdministration',
 'Metadata']

In [110]:
df_proc_all.columns.tolist()

['EHR_ID',
 'PartnerID',
 'extractionType',
 'Procedure_ID',
 'ProcedureType_ID',
 'DisplayName',
 'Side',
 'PeriodSpan',
 'Metadata',
 'ProcedureDate',
 'EvidenceCreated']

In [111]:
# ------- utilidades -------
COMMON_COLS = [
    "PartnerID", "EHR_ID", "Group", "ExtractionType",
    "Item_ID", "Item_Name", "Side",
    "Value", "Units",
    "Frequency", "FrequencyInstructions", "Duration", "StartDay", "RouteOfAdministration",
    "PeriodSpan", "ProcedureDate",
    "Metadata", "Details"   # Details: dict/JSON com extras específicos de cada grupo
]

def _safe_col(df, col, default=None):
    return df[col] if col in df.columns else default

def _ensure_cols(df, cols):
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df

def _to_jsonable(d):
    # Remove chaves com NA/None para ficar enxuto
    return {k: v for k, v in d.items() if pd.notna(v) and v is not None}

def normalize_procedures(df_proc_all):
    if df_proc_all is None or len(df_proc_all) == 0:
        return pd.DataFrame(columns=COMMON_COLS)

    out = pd.DataFrame({
        "PartnerID": _safe_col(df_proc_all, "PartnerID"),
        "EHR_ID": _safe_col(df_proc_all, "EHR_ID"),
        "Group": "procedure",
        "ExtractionType": _safe_col(df_proc_all, "extractionType"),
        # >>> AQUI: usar ProcedureType_ID como chave única p/ mapeamento <<<
        "Item_ID": _safe_col(df_proc_all, "ProcedureType_ID"),
        "Item_Name": _safe_col(df_proc_all, "DisplayName"),
        "Side": _safe_col(df_proc_all, "Side"),
        "Value": pd.NA,
        "Units": pd.NA,
        "Frequency": pd.NA,
        "FrequencyInstructions": pd.NA,
        "Duration": pd.NA,
        "StartDay": pd.NA,
        "RouteOfAdministration": pd.NA,
        "PeriodSpan": _safe_col(df_proc_all, "PeriodSpan"),
        "ProcedureDate": _safe_col(df_proc_all, "ProcedureDate"),
        "Metadata": _safe_col(df_proc_all, "Metadata"),
    })
    # Preserve Procedure_ID e outros campos em Details
    details = []
    for _, r in df_proc_all.iterrows():
        d = {
            "Procedure_ID": r.get("Procedure_ID", pd.NA),
            "ProcedureType_ID": r.get("ProcedureType_ID", pd.NA),
            "EvidenceCreated": r.get("EvidenceCreated", pd.NA),
        }
        details.append(_to_jsonable(d))
    out["Details"] = details
    return _ensure_cols(out, COMMON_COLS)

# ------- Normalizações por grupo -------
def normalize_measurements(df_meas_all):
    if df_meas_all is None or len(df_meas_all) == 0:
        return pd.DataFrame(columns=COMMON_COLS)

    out = pd.DataFrame({
        "PartnerID": _safe_col(df_meas_all, "PartnerID"),
        "EHR_ID": _safe_col(df_meas_all, "EHR_ID"),
        "Group": "measurement",
        "ExtractionType": _safe_col(df_meas_all, "extractionType"),
        "Item_ID": _safe_col(df_meas_all, "Measurement_ID"),
        "Item_Name": _safe_col(df_meas_all, "MeasurementName"),   # pode ser NA
        "Side": _safe_col(df_meas_all, "Side"),
        "Value": _safe_col(df_meas_all, "StringValue"),
        "Units": pd.NA,  # se tiver Units em algum lugar, mapeie aqui
        "Frequency": pd.NA,
        "FrequencyInstructions": pd.NA,
        "Duration": pd.NA,
        "StartDay": pd.NA,
        "RouteOfAdministration": pd.NA,
        "PeriodSpan": pd.NA,
        "ProcedureDate": pd.NA,
        "Metadata": _safe_col(df_meas_all, "Metadata"),
    })
    # Extras específicos de measurements (se quiser guardar mais coisas aqui no futuro)
    details = []
    for _, r in df_meas_all.iterrows():
        d = {}
        # exemplo: se houver colunas adicionais em df_meas_all que queira preservar, adicione aqui
        details.append(_to_jsonable(d))
    out["Details"] = details
    return _ensure_cols(out, COMMON_COLS)

def normalize_prescriptions(df_rx_all):
    if df_rx_all is None or len(df_rx_all) == 0:
        return pd.DataFrame(columns=COMMON_COLS)

    out = pd.DataFrame({
        "PartnerID": _safe_col(df_rx_all, "PartnerID"),
        "EHR_ID": _safe_col(df_rx_all, "EHR_ID"),
        "Group": "prescription",
        "ExtractionType": _safe_col(df_rx_all, "extractionType"),
        "Item_ID": _safe_col(df_rx_all, "Measurement_ID"),  # aqui é Medication_ID na origem
        "Item_Name": pd.NA,  # você não tem MedicationName, então fica NA
        "Side": _safe_col(df_rx_all, "Side"),
        "Value": _safe_col(df_rx_all, "Dosage"),            # escolha principal para "Value"
        "Units": _safe_col(df_rx_all, "MedicationDosageUnit"),
        "Frequency": _safe_col(df_rx_all, "Frequency"),
        "FrequencyInstructions": _safe_col(df_rx_all, "FrequencyInstructions"),
        "Duration": _safe_col(df_rx_all, "Duration"),
        "StartDay": _safe_col(df_rx_all, "StartDay"),
        "RouteOfAdministration": _safe_col(df_rx_all, "RouteOfAdministration"),
        "PeriodSpan": pd.NA,
        "ProcedureDate": pd.NA,
        "Metadata": _safe_col(df_rx_all, "Metadata"),
    })
    # Preserve outros campos relevantes em Details
    details = []
    for _, r in df_rx_all.iterrows():
        d = {
            "MedicationDosage": r.get("MedicationDosage", pd.NA),
            # Adicione aqui o que mais quiser preservar em JSON
        }
        details.append(_to_jsonable(d))
    out["Details"] = details
    return _ensure_cols(out, COMMON_COLS)


In [112]:
# ------- Execução: normalizar e concatenar -------
df_meas_norm = normalize_measurements(df_meas_all)
df_rx_norm   = normalize_prescriptions(df_rx_all)
df_proc_norm = normalize_procedures(df_proc_all)


In [113]:
df_meas_norm = dedup_parser_rows(
    df_meas_norm,
    id_col="Item_ID",
    name_col="Item_Name",
    value_col="Value",
    etype_col="ExtractionType",
    ehr_col="EHR_ID",
    metadata_col="Metadata",
    side_col="Side"
)


In [114]:
# --- Deduplicar apenas MEASUREMENTS do PARSER por (EHR_ID, Item_ID, Side, Value) ---
mask = (df_meas_norm["Group"].eq("measurement")) & (df_meas_norm["ExtractionType"].eq("parser"))

meas_parser = df_meas_norm.loc[mask].copy()

# normalizar Side/Value para evitar duplicatas por espaços/NaN
meas_parser["Side"]  = meas_parser["Side"].fillna("").astype(str).str.strip()
meas_parser["Value"] = meas_parser["Value"].fillna("").astype(str).str.strip()

meas_parser = meas_parser.drop_duplicates(
    subset=["EHR_ID", "Item_ID", "Side", "Value"], keep="first"
)

# recompor df_meas_norm com parser deduplicado
df_meas_norm = pd.concat([df_meas_norm.loc[~mask], meas_parser], ignore_index=True)


In [115]:
# 2) União em um único dataframe “longo”
df_all_measures = pd.concat(
    [df_meas_norm, df_rx_norm, df_proc_norm],
    ignore_index=True, sort=False
)

C:\Users\dxr1276\AppData\Local\Temp\ipykernel_15260\1642647056.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_measures = pd.concat(


In [116]:
# df_all_measures: usar Item_ID como chave
df_all_measures = df_all_measures.copy()
df_all_measures["Item_ID"] = pd.to_numeric(df_all_measures["Item_ID"], errors="coerce").astype("Int64")


In [117]:
# Carregar expectativas
with open("mapa_expectativas_nova.json", "r", encoding="utf-8") as f:
    expectations = json.load(f)

df_expect = pd.DataFrame(expectations)

def maybe_json(x):
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return x
    return x

df_expect["possible_ids"] = df_expect["possible_ids"].apply(maybe_json)

while df_expect["possible_ids"].apply(lambda v: isinstance(v, list)).any():
    df_expect = df_expect.explode("possible_ids", ignore_index=True)

In [118]:
# 0) Garantir nome do tipo
df_expect = df_expect.rename(columns={"type": "ClassificationType"})

# 1) Coagir possible_ids para numérico (placeholders viram NaN)
df_expect["possible_id"] = pd.to_numeric(df_expect["possible_ids"], errors="coerce").astype("Int64")

# 2) Tabela de mapeamento por ID (para join com medidas/Rx/procedures)
df_expect_ids = (
    df_expect
    .dropna(subset=["possible_id"])
    .loc[:, ["ClassificationType", "possible_id"]]
    .drop_duplicates()
)

# 3) Tabela de metas por tipo + lista FLAT de IDs
#    (reconstituímos a lista a partir do df_expect_ids para garantir que está achatada)
#    Primeiro, pegamos só os limites (min/med/max) do JSON original (não explodido):
df_limits = (
    pd.DataFrame(expectations)
    .rename(columns={"type": "ClassificationType"})
    [["ClassificationType", "min_expected_measurement", "median_expected_measurement", "max_expected_measurement"]]
)

#    Agora, agregamos os IDs únicos por tipo:
ids_flat = (
    df_expect_ids
    .groupby("ClassificationType", as_index=False)["possible_id"]
    .agg(lambda s: sorted([int(x) for x in s.dropna().unique()]))
    .rename(columns={"possible_id": "possible_ids"})
)

#    Por fim, juntamos limites + ids (tipos sem IDs ficam com lista vazia):
df_expect_meta = df_limits.merge(ids_flat, on="ClassificationType", how="left")
df_expect_meta["possible_ids"] = df_expect_meta["possible_ids"].apply(lambda v: v if isinstance(v, list) else [])
df_expect_meta.head()

,ClassificationType,min_expected_measurement,median_expected_measurement,max_expected_measurement,possible_ids
0,DiagnosisOphthalmological,0,0,0,[]
1,DiagnosisComorbidities,0,0,0,[]
2,FamiliarHistory,0,0,0,[]
3,PreviousOphthalmologicalProcedures,1,2,4,"[6, 7, 8, 9, 10, 11, 12, 14, 16, 19, 20, 21, 2..."
4,MedicationsInUse,1,2,4,"[8345, 8349, 8350, 8351, 8353, 8373, 9280, 932..."


In [119]:
print("#linhas em df_expect_ids:", len(df_expect_ids))
print("Exemplos de tipos e contagens de IDs:")
print(df_expect_ids["ClassificationType"].value_counts().head(20))

# Conferir se ainda sobrou algo não numérico (não deve):
sobras = df_expect.loc[df_expect["possible_id"].isna(), ["ClassificationType", "possible_ids"]].head(20)
print("Exemplos de 'possible_ids' não numéricos (ignorados no join):\n", sobras)


#linhas em df_expect_ids: 1005
Exemplos de tipos e contagens de IDs:
ClassificationType
PreviousOphthalmologicalProcedures    390
SurgicalDescription                   390
MedicationsInUse                       61
MedicationsPrescribed                  61
Refraction                             41
RefractionOnly                         41
VisualAcuity                           10
IntraocularPressure                     9
Cid                                     2
Name: count, dtype: int64
Exemplos de 'possible_ids' não numéricos (ignorados no join):
                    ClassificationType possible_ids
0           DiagnosisOphthalmological          NaN
1              DiagnosisComorbidities          NaN
2                     FamiliarHistory          NaN
515                          SlitLamp          NaN
516   NeurossensorialOcularEvaluation          NaN
579                    MedicalConduct          NaN
621                 ExaminationReport          NaN
1012                 AnesthesiaRecord

In [120]:
# Unificar extrações normalizadas
df_all_measures = pd.concat([df_meas_norm, df_rx_norm, df_proc_norm], ignore_index=True, sort=False)

# Garantir tipagem da chave
df_all_measures["Item_ID"] = pd.to_numeric(df_all_measures["Item_ID"], errors="coerce").astype("Int64")

# Join por ID para obter a ClassificationType de cada linha extraída
measures_with_class = df_all_measures.merge(
    df_expect_ids.rename(columns={"possible_id": "Item_ID"})[["ClassificationType", "Item_ID"]],
    on="Item_ID",
    how="inner"
)


C:\Users\dxr1276\AppData\Local\Temp\ipykernel_15260\3228459044.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_measures = pd.concat([df_meas_norm, df_rx_norm, df_proc_norm], ignore_index=True, sort=False)


In [121]:
# Contagens unificadas (measurement + prescription + procedure)
df_counts_all = (
    measures_with_class
    .groupby(["PartnerID", "ClassificationType", "ExtractionType"])
    .size()
    .reset_index(name="n")
    .pivot(index=["PartnerID", "ClassificationType"], columns="ExtractionType", values="n")
    .reset_index()
    .fillna(0)
    .rename(columns={"llm": "Medidas_llm", "parser": "Medidas_parser"})
)

# Garantir colunas mesmo se alguma origem não aparecer
for c in ["Medidas_llm", "Medidas_parser"]:
    if c not in df_counts_all.columns:
        df_counts_all[c] = 0

# (check rápido)
df_counts_all.head()

ExtractionType,PartnerID,ClassificationType,Medidas_llm,Medidas_parser
0,NP:1094013_APAC:3087727_MR:1288344,PreviousOphthalmologicalProcedures,0.0,8.0
1,NP:1094013_APAC:3087727_MR:1288344,Refraction,6.0,10.0
2,NP:1094013_APAC:3087727_MR:1288344,RefractionOnly,6.0,10.0
3,NP:1094013_APAC:3087727_MR:1288344,SurgicalDescription,0.0,8.0
4,NP:1094013_APAC:3087727_MR:1288347,MedicationsInUse,1.0,1.0


In [122]:
# base = 1 linha por (EHR, ClassificationType) presente no Contains + metas do JSON
# Pré: ehr_flags tem ["EHR_ID","Patient_ID","PartnerID","EHR_Date","FormattedText","Contains","ClassificationType"]
#      df_expect_meta tem ["ClassificationType","min_expected_measurement","median_expected_measurement",
#                          "max_expected_measurement","possible_ids"]
base = ehr_flags.merge(df_expect_meta, on="ClassificationType", how="left")

# juntar contagens (measurement + prescription + procedure) por PartnerID+ClassificationType
df_final = base.merge(
    df_counts_all, on=["PartnerID","ClassificationType"], how="left"
)

# garantir colunas e tipos
for c in ["Medidas_llm","Medidas_parser"]:
    if c not in df_final.columns: df_final[c] = 0
df_final[["Medidas_llm","Medidas_parser"]] = df_final[["Medidas_llm","Medidas_parser"]].fillna(0).astype(int)

# Total e metas
df_final["Total"] = df_final["Medidas_llm"] + df_final["Medidas_parser"]
for c in ["min_expected_measurement","median_expected_measurement","max_expected_measurement"]:
    df_final[c] = pd.to_numeric(df_final[c], errors="coerce").fillna(0).astype(int)

df_final["Atingiu_minimo"]  = df_final["Total"] >= df_final["min_expected_measurement"]
df_final["Atingiu_mediana"] = df_final["Total"] >= df_final["median_expected_measurement"]
df_final["Atingiu_maximo"]  = df_final["Total"] >= df_final["max_expected_measurement"]

# ‘type’ = alias para compatibilidade e ordenar colunas chave
df_final["type"] = df_final["ClassificationType"]
df_final = df_final.sort_values(["Patient_ID","EHR_ID","ClassificationType"])
df_final.head()


,EHR_ID,Patient_ID,PartnerID,EHR_Date,FormattedText,Contains,ClassificationType,min_expected_measurement,median_expected_measurement,max_expected_measurement,possible_ids,Medidas_llm,Medidas_parser,Total,Atingiu_minimo,Atingiu_mediana,Atingiu_maximo,type
129,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,DiagnosisComorbidities,0,0,0,[],0,0,0,True,True,True,DiagnosisComorbidities
1024,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,MedicalConduct,0,0,0,[],0,0,0,True,True,True,MedicalConduct
898,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,Refraction,1,6,12,"[52, 53, 54, 55, 56, 57, 58, 475, 476, 477, 48...",8,8,16,True,True,True,Refraction
773,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,VisualAcuity,1,2,8,"[44, 45, 46, 123655, 123744, 123745, 123755, 1...",2,2,4,True,True,False,VisualAcuity
899,4641017,280530,NP:664586_APAC:1506666_MR:386059,2016-11-16 13:41:07,Receita\r\nID da Receita: 386059\r\nID do Aten...,9216,Refraction,1,6,12,"[52, 53, 54, 55, 56, 57, 58, 475, 476, 477, 48...",6,10,16,True,True,True,Refraction


In [123]:
def to_json_safe(df_rows: pd.DataFrame, cols):
    out = []
    for row in df_rows.to_dict(orient="records"):
        item = {}
        for c in cols:
            if c in row and pd.notnull(row[c]):
                v = row[c]
                # timestamps em ISO
                if isinstance(v, pd.Timestamp):
                    v = v.isoformat()
                item[c] = v
        out.append(item)
    return json.dumps(out, ensure_ascii=False)

# 1) Detalhes de MEASUREMENTS
meas_grouped = (
    df_meas_all.groupby(["PartnerID","extractionType"], dropna=False)
    .apply(lambda g: to_json_safe(
        g,
        ["EHR_ID","extractionType","Measurement_ID","MeasurementName","Side","StringValue","Metadata"]
    ))
    .unstack(fill_value="[]")
    .reset_index()
    .rename(columns={"llm":"measurements_llm","parser":"measurements_parser"})
)

# 2) Detalhes de PRESCRIPTIONS
rx_grouped = (
    df_rx_all.groupby(["PartnerID","extractionType"], dropna=False)
    .apply(lambda g: to_json_safe(
        g,
        ["EHR_ID","extractionType","Measurement_ID","MedicationName","Side",
         "Frequency","FrequencyInstructions","Duration","StartDay",
         "MedicationDosage","Dosage","MedicationDosageUnit","RouteOfAdministration","Metadata"]
    ))
    .unstack(fill_value="[]")
    .reset_index()
    .rename(columns={"llm":"prescriptions_llm","parser":"prescriptions_parser"})
)

# 3) Detalhes de PROCEDURES
proc_grouped = (
    df_proc_all.groupby(["PartnerID","extractionType"], dropna=False)
    .apply(lambda g: to_json_safe(
        g,
        ["EHR_ID","extractionType","Procedure_ID","ProcedureType_ID","DisplayName","Side",
         "PeriodSpan","Metadata","ProcedureDate","EvidenceCreated"]
    ))
    .unstack(fill_value="[]")
    .reset_index()
    .rename(columns={"llm":"procedures_llm","parser":"procedures_parser"})
)

# 4) Mesclar os detalhes no df_final
df_final = (
    df_final
    .merge(meas_grouped, on="PartnerID", how="left")
    .merge(rx_grouped,   on="PartnerID", how="left")
    .merge(proc_grouped, on="PartnerID", how="left")
)

# Garantir colunas e preencher ausências
for c in ["measurements_llm","measurements_parser",
          "prescriptions_llm","prescriptions_parser",
          "procedures_llm","procedures_parser"]:
    if c not in df_final.columns:
        df_final[c] = "[]"
    else:
        df_final[c] = df_final[c].fillna("[]")

print("OK: df_final agora contém as colunas de detalhes JSON.")


OK: df_final agora contém as colunas de detalhes JSON.


In [124]:
df_final.head()

,EHR_ID,Patient_ID,PartnerID,EHR_Date,FormattedText,Contains,ClassificationType,min_expected_measurement,median_expected_measurement,max_expected_measurement,...,Atingiu_minimo,Atingiu_mediana,Atingiu_maximo,type,measurements_llm,measurements_parser,prescriptions_llm,prescriptions_parser,procedures_llm,procedures_parser
0,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,DiagnosisComorbidities,0,0,0,...,True,True,True,DiagnosisComorbidities,"[{""EHR_ID"": 4641016, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6233596, ""extractionType"": ""parser...",[],[],[],[]
1,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,MedicalConduct,0,0,0,...,True,True,True,MedicalConduct,"[{""EHR_ID"": 4641016, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6233596, ""extractionType"": ""parser...",[],[],[],[]
2,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,Refraction,1,6,12,...,True,True,True,Refraction,"[{""EHR_ID"": 4641016, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6233596, ""extractionType"": ""parser...",[],[],[],[]
3,4641016,280530,NP:664586_APAC:1506666_OCM:1096443,2016-11-16 13:41:07,Consulta Médica\r\nID da Consulta: 1096443\r\n...,5634,VisualAcuity,1,2,8,...,True,True,False,VisualAcuity,"[{""EHR_ID"": 4641016, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6233596, ""extractionType"": ""parser...",[],[],[],[]
4,4641017,280530,NP:664586_APAC:1506666_MR:386059,2016-11-16 13:41:07,Receita\r\nID da Receita: 386059\r\nID do Aten...,9216,Refraction,1,6,12,...,True,True,True,Refraction,"[{""EHR_ID"": 4641017, ""extractionType"": ""llm"", ...","[{""EHR_ID"": 6233618, ""extractionType"": ""parser...",[],[],[],[]


In [125]:
df_final.to_csv("df_final3.csv", index=False)